In [1]:
import numpy as np
import cv2

%pylab inline 

%load_ext Cython

Populating the interactive namespace from numpy and matplotlib


In [2]:
%%cython
# Use %%cython -a for more verbose output, note %%cython should be on the first line in the cell!
def get_8_connected(int x, int y, shape):
    cdef int xmax = shape[0]-1
    cdef int ymax = shape[1]-1
    cdef int connected_pixel_x 
    cdef int connected_pixel_y
    cdef int dx, dy
    
    connected_pixels = list()
    
    for dx in range(3):
        for dy in range(3):
            connected_pixel_x  = x + dx - 1
            connected_pixel_y = y + dy - 1
            if connected_pixel_x < 0 or connected_pixel_x > xmax or \
                connected_pixel_y < 0 or connected_pixel_y > ymax or \
                (connected_pixel_x == x and connected_pixel_y == y):
                    pass
            else: 
                connected_pixels.append((connected_pixel_x,connected_pixel_y)) 
    return  connected_pixels

In [7]:
def region_growing(img, 
                   seed_points, 
                   test = lambda seed_x, seed_y, x, y, img, outimg : img[x,y] != 0,
                   colormap=None):
    
    processed = np.full((img.shape[0],img.shape[1]), False)
    
    if colormap is None:
        outimg = np.zeros_like(img)
    else:
        outimg = np.zeros((img.shape[0],img.shape[1],colormap.shape[1]),dtype=np.uint8)
    
    for index, pix in enumerate(seed_points):
        processed[pix[0], pix[1]] = True
        if colormap is None:
            outimg[pix[0], pix[1]] = img[pix[0], pix[1]]
        else:
            outimg[pix[0], pix[1]]  = colormap[index % len(colormap)]
    
    while(len(seed_points) > 0):
        pix = seed_points[0]
            
        for coord in get_8_connected(pix[0], pix[1], img.shape):
            if not processed[coord[0],coord[1]]:
                test_result = test(pix[0], pix[1], coord[0], coord[1], img, outimg)
                if test_result:
                    outimg[coord[0], coord[1]]  = outimg[pix[0], pix[1]]
                    if not processed[coord[0],coord[1]]:
                        seed_points.append(coord)
                    processed[coord[0],coord[1]] = True
        #Process Animation
        cv2.imshow("progress",outimg)
        cv2.waitKey(1)           
        
        seed_points.pop(0)
    return outimg

In [11]:
def on_mouse(event, x, y, flags, params):
    if event == cv2.EVENT_LBUTTONDOWN:
        print( 'Seed: ' + str(x) + ', ' + str(y), img[y,x])
        clicks.append((int(y), int(x)))
        
clicks = []
image = cv2.imread('Test2.bmp', 0)
image = cv2.cvtcolor(image, cv2.COLOR_BGR2GRAY)
ret, img = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY)
cv2.namedWindow('Input')
cv2.setMouseCallback('Input', on_mouse, 0, )
cv2.imshow('Input', img)
cv2.waitKey(0)
#cv2.destroyAllWindows()

seed = clicks
out = region_growing(img, seed)
cv2.imshow('Region Growing', out)
cv2.waitKey(0)
cv2.destroyAllWindows()

AttributeError: module 'cv2' has no attribute 'cvtcolor'